In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys, os
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import Dataset
from torchvision import transforms
import random
from omegaconf import OmegaConf
from typing import Iterable, Dict, Callable, Tuple, Union, List
from scipy.ndimage import binary_erosion, binary_dilation
import random
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import binary_erosion, binary_dilation


sys.path.append('..')
from dataset import get_eval_dataset
from utils import get_features, evaluate_RF
from model import get_model
from user_model import UserModel

/tmp/ipykernel_2226109/3993998618.py:13: DeprecationWarning: Please import `binary_erosion` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.morphology import binary_erosion, binary_dilation
/tmp/ipykernel_2226109/3993998618.py:13: DeprecationWarning: Please import `binary_dilation` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.morphology import binary_erosion, binary_dilation


### Boiler plate

In [14]:
# Load config
cfg = OmegaConf.load('../configs/eval_uncertainty.yaml')

cfg.feature = False
cfg.uncertainty_measure = 'ground-truth'
cfg.soft_scores = True

In [5]:
verbose = True

# get dataset
dataset = get_eval_dataset(
    cfg=cfg,
    initial_annotation=True,
    verbose=verbose
)

# get model and load state dict
model, state_dict = get_model(
    cfg=cfg,
    return_state_dict=True,
    verbose=verbose
)
model.load_state_dict(state_dict)

# get features
features = get_features(
    model=model,
    dataset=dataset,
    feature=cfg.feature,
    verbose=verbose
)

# get results
uncertainty_measures = [
    cfg.uncertainty_measure
]
if cfg.background_bias:
    uncertainty_measures.append('feature-distance')

Loading model: zero -> reconstruction with spatial dim 145 and Dropout False

Done. Returning model and state dict ae_feature-extractor_all_old.pt.

Extracting features...
Done.



In [32]:

# results of initial annotation
scores, prediction, uncertainty_maps, uncertainty_per_class_maps, t = evaluate_RF(
    dataset=dataset, 
    features=features, 
    cfg=cfg,
    uncertainty_measures=uncertainty_measures
)

In [84]:
dataset.user = UserModel(dataset.label, 'uniform', soft_scores=cfg.soft_scores, cfg=cfg)

uncertainty_measure = uncertainty_measures[0]
u_annots, _ = dataset.user.refinement_annotation(
    prediction=prediction,
    annotation_mask=dataset.annotations.detach().cpu(),
    uncertainty_map=uncertainty_per_class_maps[uncertainty_measure] if uncertainty_measure != 'ground-truth' else None,
    n_samples=200,
    mode='per_class',
    seed=42,
    inverse_class_freq=False
)

2 45
0 95
1 45
1 66
0 22


In [69]:
torch.zeros((10,10)).numel()

100

In [20]:
dataset.user = UserModel(dataset.label, 'uniform', cfg)
dataset.init = 'per_class'
# currently, there are no annotations. We can also enforce this with clear_annotations() at any point
dataset.clear_annotation()
# get initial annotations
annot = dataset.initial_annotation(seed=42)
# and update the dataset
dataset.update_annotation(annot)
print(f"number of annotations: {dataset.annotations.detach().cpu().sum()}")
cached_annots = dataset.annotations.clone()

TypeError: UserModel.__init__() missing 1 required positional argument: 'cfg'